### Getting Date Time Objects

In [2]:
import pandas as pd
from datetime import datetime, timedelta

In [3]:
bookmarks_csv = pd.read_csv('../Torah-Navigator/bookmarks_cleaned.csv')
categories_csv = pd.read_csv('../Torah-Navigator/categories_cleaned.csv')
favorites_csv = pd.read_csv('../Torah-Navigator/favorites_cleaned.csv')
shiurim_csv = pd.read_csv('../Torah-Navigator/shiurim_cleaned.csv')
shiurim_csv['date'] = pd.to_datetime(shiurim_csv['date'], format='mixed')
favorites_csv['date_favorite_added'] = pd.to_datetime(favorites_csv['date_favorite_added'], format='mixed')
bookmarks_csv['date_played'] = pd.to_datetime(bookmarks_csv['date_played'], format='mixed')
bookmarks_csv['date_downloaded'] = pd.to_datetime(bookmarks_csv['date_downloaded'], format='mixed')
bookmarks_csv['queue_date'] = pd.to_datetime(bookmarks_csv['queue_date'], format='mixed')

In [ ]:
bookmarks_csv.dtypes

In [ ]:
df = bookmarks_csv.dropna(subset='date_played')
df.head()

Check Csvs are loaded

In [ ]:
bookmarks_csv

In [ ]:
def remove_microseconds(date_str):
    return date_str.split('.')[0]

In [ ]:
# Drop NaN values and sort by 'date_played'
bookmarks_clean = bookmarks_csv.dropna(subset=['date_played']).sort_values(by='date_played', ascending=False)

# Apply function to 'date_played' column
#bookmarks_clean['date_played'] = bookmarks_clean['date_played'].apply(remove_microseconds)

# Convert 'date_played' to datetime
#bookmarks_clean['date_played'] = pd.to_datetime(bookmarks_clean['date_played'], format='%Y-%m-%d %H:%M:%S')
bookmarks_clean

In [ ]:
most_recent_listen = bookmarks_clean['date_played'].max()
most_recent_listen

In [ ]:
one_week_before = most_recent_listen - timedelta(weeks=1)
one_week_before

In [ ]:
recent_bookmarks = bookmarks_clean[bookmarks_clean['date_played'] >= one_week_before]
recent_bookmarks

In [ ]:
class general_recommendations(BaseModel):
    def get_trending(self) -> pd.DataFrame:
        

In [ ]:
def __get_most_recent_shiurim(bookmarks_df: pd.DataFrame, recent = 7):
    

In [ ]:
print(bookmarks_csv.isna().all())

In [ ]:
print(type(favorites_csv['date_favorite_added'].max()))
print(type(shiurim_csv['date'].values[0]))
print(type(bookmarks_csv['date_played'].values[0]))
print(type(bookmarks_csv['queue_date'].values[0]))
print(type(bookmarks_csv['date_downloaded'].values[0]))

In [ ]:
shiurim_csv.dtypes

In [ ]:
# Get the data types of each element in the column
data_types = shiurim_csv['date'].apply(type)

# Check unique data types
unique_data_types = data_types.unique()

print("Data types present in 'column1':")
for dtype in unique_data_types:
    print(dtype)


In [ ]:
def __remove_microseconds(date_str: str):
        return date_str.split('.')[0]
    
def __convert_to_datetime(column: pd.Series):
    column = column.apply(__remove_microseconds)  # Apply __remove_microseconds first
    column = pd.to_datetime(column, format='%Y-%m-%d %H:%M:%S')  # Convert to datetime
    return column

In [ ]:
favorites_csv['date_favorite_added'] = __convert_to_datetime(favorites_csv['date_favorite_added'])
favorites_csv

In [ ]:
print(type(favorites_csv['date_favorite_added'].max()))

In [ ]:
favorites_csv['date_favorite_added'] = pd.to_datetime(
    favorites_csv['date_favorite_added'], format='mixed').dt.floor('s')

In [ ]:
print(type(shiurim_csv['date'].max()))

In [ ]:
shiurim_csv['date'] = pd.to_datetime(shiurim_csv['date'], format='mixed').dt.floor('S')

In [ ]:
print(type(shiurim_csv['date'].max()))

In [ ]:
print(type(bookmarks_csv['queue_date'].max()))

In [ ]:
bookmarks_csv = bookmarks_csv.dropna(subset=['queue_date'])

In [ ]:
import sys
import os

# Determine the relative path to the src directory
relative_path_to_src = os.path.abspath(os.path.join(os.getcwd(), '../Torah-Navigator'))

# Add the src directory to the system path
if relative_path_to_src not in sys.path:
    sys.path.append(relative_path_to_src)

from src.pipeline.data_processor import DataProcessor, CleanedData

dp = DataProcessor()

df = dp.load_table(CleanedData.SHIURIM) # Other options: CleanedData.BOOKMARKS, CleanedData.FAVORITES, CleanedData.CATEGORIES

In [ ]:
print(type(df['date'].iloc[1000]))

In [ ]:
dp.load_limit_table(CleanedData.CATEGORIES, 100).dtypes

### Requirements For Converting Dates With YUTorah data

- .astype('datetime64[ns]') 
- parse_dates=['date_column_name']
    - This is needed for csv files, untested if needed for loading from database
- errors='coerce'
- not working within same column

In [ ]:
def __get_date_columns(table_name: str) -> list:
        if table_name == "shiurim_cleaned":
            return ['date']
        elif table_name == 'bookmarks_cleaned':
            parse_dates = ['date_played', 'date_downloaded', 'queue_date']
        elif table_name == 'favorites_cleaned':
            parse_dates = ['date_favorite_added']
        else:
            return []

In [ ]:
print(__get_date_columns(CleanedData.SHIURIM))

In [ ]:
def __extract_table_name(self, query: str) -> str:
        # Simple parsing assuming a specific query format: 'SELECT * FROM table_name'
        pattern = r'FROM\s+(\w+)'
        match = re.search(pattern, query, re.IGNORECASE)
        if match:
            return match.group(1)
        else:
            return None
        
        ### Method for finding table names in query

### Getting Trending Shiurim

In [ ]:
from datetime import date

print(date.today())

In [ ]:
def get_recent_interactions(bookmarks : pd.DataFrame, past: int = 7):
    table = bookmarks
    today = pd.to_datetime(date.today())
    delta = today - timedelta(days=past)
    table = table[(table['date_played'] > delta) | (table['date_downloaded'] > delta) | (table['queue_date'] > delta)]
    return table

In [ ]:
get_recent_interactions(bookmarks_csv, 41)

In [ ]:
def get_top_recent_shiurim(table: pd.DataFrame,shiurim: pd.DataFrame, past: int = 7, category: str = None) -> pd.DataFrame:
    # There are only 32 total downloads from cleaned data so no seprate column for downloads
    a = pd.DataFrame()
    df = get_recent_interactions(table, past)
    played = df[df['played'] == 1].groupby('shiur').size().reset_index(name='total_listens')
    queued = df.dropna(subset='queue_date').groupby('shiur')
    #a['total_listens'] = played['played'].count()
    a['total_queues'] = queued['queue_date'].count()

    #if category is not None and category in shiurim['category'].values:
     #   a = a[a['category'] == category]
    
    return played.sort_values(by='total_listens', ascending=False)


In [ ]:
a = pd.DataFrame()
df = get_recent_interactions(bookmarks_csv,151)
played = df[df['played'] == 1]
grouped = played.groupby('shiur').size()
a['total_listens'] = grouped
a.sort_values(by='total_listens', ascending=False)

In [ ]:
b = pd.DataFrame()
df2 = get_recent_interactions(bookmarks_csv, 48)
#df2 = df2[df2['played'] == 1]
grouped = df2.groupby('shiur').size()
b['total_interactions'] = grouped
b.sort_values(by='total_interactions', ascending=False)

In [ ]:
normalize_popularity(get_top_recent_shiurim(bookmarks_csv, 45)).sort_values(by='popularity', ascending=False)

In [ ]:
get_top_recent_shiurim(bookmarks_csv,45).sort_values(by='total_listens', ascending = False)

In [ ]:
normalize_popularity(get_top_recent_shiurim(bookmarks_csv, 75)).sort_values(by='total_listens', ascending=False)

In [ ]:
def normalize_popularity(df : pd.DataFrame):
    df['normalized_listens'] = (df['total_listens'] - df['total_listens'].min()) / (df['total_listens'].max() - df['total_listens'].min())
    #df['normalized_queues'] = (df['total_queues'] - df['total_queues'].min()) / (df['total_queues'].max() - df['total_queues'].min())

    weight_listens = 0.7
    weight_queues = 0.3

    #df['popularity'] = (weight_listens * df['normalized_listens']) + (weight_queues * df['normalized_queues'])

    #df.drop(columns=['normalized_listens', 'normalized_queues'], inplace=True)

    return df

In [ ]:
def __get_shiur_listens(shiurim: pd.DataFrame, bookmarks: pd.DataFrame) -> pd.DataFrame:
    filtered = bookmarks[bookmarks['played'] == 1]
    listens_df = filtered.groupby('shiur').size().reset_index(name='listens')
    merged_df = pd.merge(shiurim, listens_df, on='shiur', how='left')
    merged_df['listens'] = merged_df['listens'].fillna(0).astype(int)  # Fill NaNs with 0 and convert to int
    return merged_df

In [ ]:
__get_shiur_listens(shiurim_csv,bookmarks_csv).sort_values(by='listens', ascending=False)

### Testing

In [ ]:
from datetime import date, timedelta

In [ ]:
def __get_recent_interactions(bookmarks : pd.DataFrame, past: int = 7) -> pd.DataFrame:
        interactions = bookmarks
        today = pd.to_datetime(date.today())
        delta = today - timedelta(days=past)
        interactions = interactions[(interactions['date_played'] > delta) |
                                    (interactions['date_downloaded'] > delta) | 
                                    (interactions['queue_date'] > delta)
                                    ]
        return interactions

In [ ]:
def __get_top_recent_shiurim(bookmarks: pd.DataFrame, past: int = 7) -> pd.DataFrame:
        shiurim = pd.DataFrame()
        interactions = __get_recent_interactions(bookmarks, past)

        played = interactions[interactions['played'] == 1].groupby('shiur')
        queued = interactions.dropna(subset='queue_date').groupby('shiur')

        shiurim['total_listens'] = played['played'].count()
        shiurim['total_queues'] = queued['queue_date'].count()
        shiurim = shiurim.fillna(0)
        shiurim['total_interactions'] = shiurim['total_listens'] + shiurim['total_queues']
    
        return shiurim.sort_values(by='total_interactions', ascending=False)

In [ ]:
def __get_popularity(shiurim : pd.DataFrame) -> pd.DataFrame:
        shiurim['normalized_listens'] = (
            (shiurim['total_listens'] - shiurim['total_listens'].min()) / 
            (shiurim['total_listens'].max() - shiurim['total_listens'].min())
        )
        shiurim['normalized_queues'] = (
            (shiurim['total_queues'] - shiurim['total_queues'].min()) / 
            (shiurim['total_queues'].max() - shiurim['total_queues'].min())
        )

        weight_listens = 0.7
        weight_queues = 0.3

        shiurim['popularity'] = (
            (weight_listens * shiurim['normalized_listens']) + 
            (weight_queues * shiurim['normalized_queues'])
        )

        shiurim.drop(columns=['normalized_listens', 'normalized_queues'], inplace=True)

        return shiurim.sort_values(by='popularity', ascending=False)

In [ ]:
__get_recent_interactions(bookmarks_csv, 48)

In [ ]:
__get_top_recent_shiurim(bookmarks_csv, 55)

In [ ]:
df = __get_popularity(__get_top_recent_shiurim(bookmarks_csv, 55))
df

In [ ]:
print(df.index[:5].tolist())

In [ ]:
list(zip(df.index[:5], df['popularity'][:5]))

### Categories

In [ ]:
shiurim = shiurim_csv[['shiur','title','teacher_title','last_name','first_name','duration','category','middle_category','subcategory','series_name']]
df2 = pd.merge(df,shiurim, on='shiur')
df2


In [ ]:
df2['name'] = df2['teacher_title'] + ' ' + df2['first_name'] + ' ' + df2['last_name']
df2.drop(columns=['teacher_title', 'first_name','last_name'], inplace=True)
df2

### FILTER

In [ ]:
df2 = df2[df2['duration'] < 1000]
df2

In [ ]:
shiurim_csv
shiurim_csv['name'] = shiurim_csv['teacher_title'] + ' ' + shiurim_csv['first_name'] + ' ' + shiurim_csv['last_name']
shiurim_csv

In [ ]:
def __merge_shiurim(shiurim : pd.DataFrame, popularity : pd.DataFrame) -> pd.DataFrame:
    shiurim['name'] = shiurim['teacher_title'] + ' ' + shiurim['first_name'] + ' ' + shiurim['last_name']
    shiurim.drop(columns=['teacher_title', 'first_name','last_name'], inplace=True)
    shiurim = shiurim[['shiur','title','name','duration','category','middle_category','subcategory','series_name']]
    merged = pd.merge(popularity, shiurim, on='shiur')
    return merged

In [ ]:
m = __merge_shiurim(shiurim_csv, __get_popularity(__get_top_recent_shiurim(bookmarks_csv, 55)))
m

In [ ]:
m

In [ ]:
def __get_filtered(merged_df : pd.DataFrame, **kwargs) -> pd.DataFrame:
    filtered_df = merged_df.copy()
    for key, value in kwargs.items():
        filtered_df = filtered_df[filtered_df[key] == value]
    return filtered_df

In [ ]:
__get_filtered(m, category='Parsha')

### Real Testing

In [ ]:
from typing import Dict, List
from .base import BaseModel
import pandas as pd
from datetime import date, timedelta
from src.pipeline.data_processor import DataProcessor, CleanedData


class Trending(BaseModel):
    def __init__(self):
        super().__init__()
        dp = DataProcessor()
        self.bookmarks = dp.load_table(CleanedData.BOOKMARKS)
        self.shiurim = dp.load_table(CleanedData.SHIURIM)
        self.merged = self.__merge_shiurim(self.shiurim,self.bookmarks)

    def get_recommendations(self, user_id: str = None, top_n: int = 5, past: int = 7, filter: str = None, *args, **kwargs) -> List[int]:
        for key,value in kwargs.items():
            if key in self.shiurim.columns:
                trending_filtered =  self.__get_filtered(key=value)
                return self.__get_popularity(trending_filtered,past).index[:top_n].tolist()
        trending_shiurim = self.__get_popularity(
            self.__get_top_recent_shiurim(self.bookmarks, past))
        return trending_shiurim.index[:top_n].tolist()

    def get_weighted_recommendations(self, user_id: str = None, top_n: int = 5, past: int = 7, *args, **kwargs) -> Dict[int, float]:
        trending_shiurim = self.__get_popularity(
            self.__get_top_recent_shiurim(self.bookmarks, past))
        return dict(zip(trending_shiurim.index[:top_n], trending_shiurim['popularity'][:top_n]))

    def __get_recent_interactions(self, bookmarks: pd.DataFrame, past: int = 7) -> pd.DataFrame:
        interactions = bookmarks
        today = pd.to_datetime(date.today())
        delta = today - timedelta(days=past)
        interactions = interactions[(interactions['date_played'] > delta) |
                                    (interactions['date_downloaded'] > delta) |
                                    (interactions['queue_date'] > delta)
                                    ]
        return interactions

    def __get_top_recent_shiurim(self, bookmarks: pd.DataFrame, past: int = 7) -> pd.DataFrame:
        shiurim = pd.DataFrame()
        interactions = self.__get_recent_interactions(bookmarks, past)

        played = interactions[interactions['played'] == 1].groupby('shiur')
        queued = interactions.dropna(subset='queue_date').groupby('shiur')

        shiurim['total_listens'] = played['played'].count()
        shiurim['total_queues'] = queued['queue_date'].count()
        shiurim = shiurim.fillna(0)
        shiurim['total_interactions'] = shiurim['total_listens'] + \
            shiurim['total_queues']

        return shiurim.sort_values(by='total_interactions', ascending=False)

    def __get_popularity(self, shiurim: pd.DataFrame) -> pd.DataFrame:
        shiurim['normalized_listens'] = (
            (shiurim['total_listens'] - shiurim['total_listens'].min()) /
            (shiurim['total_listens'].max() - shiurim['total_listens'].min())
        )
        shiurim['normalized_queues'] = (
            (shiurim['total_queues'] - shiurim['total_queues'].min()) /
            (shiurim['total_queues'].max() - shiurim['total_queues'].min())
        )

        weight_listens = 0.7
        weight_queues = 0.3

        shiurim['popularity'] = (
            (weight_listens * shiurim['normalized_listens']) +
            (weight_queues * shiurim['normalized_queues'])
        )

        shiurim.drop(columns=['normalized_listens',
                     'normalized_queues'], inplace=True)

        return shiurim.sort_values(by='popularity', ascending=False)

    def __merge_shiurim(self, shiurim: pd.DataFrame, bookmarks: pd.DataFrame) -> pd.DataFrame:
        shiurim['name'] = shiurim['teacher_title'] + ' ' + \
            shiurim['first_name'] + ' ' + shiurim['last_name']
        shiurim.drop(columns=['teacher_title',
                     'first_name', 'last_name'], inplace=True)
        shiurim = shiurim[['shiur', 'title', 'name', 'duration',
                           'category', 'middle_category', 'subcategory', 'series_name']]
        merged = pd.merge(bookmarks, shiurim, on='shiur')
        return merged

    def __get_filtered(self, **kwargs) -> pd.DataFrame:
        trending_shiurim = self.__get_top_recent_shiurim(self.merged)
        for key, value in kwargs.items():
            trending_shiurim = trending_shiurim[trending_shiurim[key] == value]
        return trending_shiurim


In [4]:
shiurim = shiurim_csv.copy().sort_values(by='shiur')
bookmarks = bookmarks_csv.copy().sort_values(by='shiur')

In [5]:
shiurim['name'] = shiurim['teacher_title'] + ' ' + \
            shiurim['first_name'] + ' ' + shiurim['last_name']
shiurim.drop(columns=['teacher_title',
                     'first_name', 'last_name'], inplace=True)
shiurim = shiurim[['shiur', 'title', 'name',
                           'category', 'middle_category', 'subcategory', 'series_name','full_details']]
merged = pd.merge(bookmarks, shiurim, on='shiur',how='left')

In [ ]:
sf = shiurim.dropna(subset=['shiur','title'])
sf

In [ ]:
bookmarks.dropna(subset='shiur')

In [ ]:
df = merged.dropna(subset=['shiur','title'])
df

In [10]:
merged.dropna(subset=['shiur','title'])

,Unnamed: 0,user,shiur,session,bookmark,timestamp,queue_date,played,date_played,downloaded,date_downloaded,duration,listen_percentage,title,name,category,middle_category,subcategory,series_name,full_details
0,3227,14160,703702,YUTORAH_NEWSESSIONAPP_38840678_55842142,lastPlayed,31.6,NaT,0,NaT,0,NaT,4731.0,0.007,hilchos nidda,Rabbi Shalom Rosner,Gemara,General,General,NaN,Title hilchos nidda Speaker Rosner Category Ge...
1,3226,28139,703751,YUTORAH_NEWSESSIONAPP_45593954_74468168,lastPlayed,98.5,NaT,0,NaT,0,NaT,1299.0,0.076,Hesped for Rabbi Israel Miller,Rabbi Dovid Miller,Personalities,General,General,NaN,Title Hesped for Rabbi Israel Miller Speaker M...
2,3225,7066,703884,YUTORAH_NEWSESSIONAPP_42294627_55600362,lastPlayed,39.7,NaT,0,NaT,0,NaT,4353.0,0.009,Ketuvot 13,Rabbi Yonason Sacks,Gemara,Nashim,Ketuvot,Daily Shiur,Title Ketuvot 13 Speaker Sacks Category Gemara
3,3224,7066,703890,YUTORAH_NEWSESSIONAPP_42294627_55600362,lastPlayed,47.3,NaT,0,NaT,0,NaT,3549.0,0.013,Ketuvot 14,Rabbi Yonason Sacks,Gemara,Nashim,Ketuvot,Daily Shiur,Title Ketuvot 14 Speaker Sacks Category Gemara
4,3223,7066,703891,YUTORAH_NEWSESSIONAPP_42294627_55600362,lastPlayed,167.6,NaT,0,NaT,0,NaT,4177.0,0.040,Ketuvot 15,Rabbi Yonason Sacks,Gemara,Nashim,Ketuvot,Daily Shiur,Title Ketuvot 15 Speaker Sacks Category Gemara
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,2423,5515,749198,YUTORAH_NEWSESSIONAPP_35297109_62401042,lastPlayed,742.6,NaT,0,NaT,0,NaT,7343.0,0.101,Parshas Kedoshim,Rabbi Joseph B Soloveitchik,Parsha,Vayikra,Kedoshim,NaN,Title Parshas Kedoshim Speaker Soloveitchik Ca...
805,2422,26916,749252,YUTORAH_NEWSESSIONAPP_43024027_53342571,lastPlayed,1320.5,NaT,0,NaT,0,NaT,2245.0,0.588,Yom Kippur Succos and Simchat Torah for Children,Rebbetzin Tziporah Heller,Machshava,Holidays,Elul,Naaleh,Title Yom Kippur Succos and Simchat Torah for ...
806,2421,10805,749358,YUTORAH_NEWSESSIONAPP_36773449_50674061,lastPlayed,164.5,NaT,0,NaT,0,NaT,777.0,0.212,Ten Minute Halacha Shnayim Mikra,Rabbi Aryeh Lebowitz,Halacha,Miscellaneous,Torah,Ten Minute Halacha,Title Ten Minute Halacha Shnayim Mikra Speake...
807,2420,10805,749408,YUTORAH_NEWSESSIONAPP_36773449_50674061,lastPlayed,163.4,NaT,0,NaT,0,NaT,648.0,0.252,Ten Minute Halacha Greetings Before Davening,Rabbi Aryeh Lebowitz,Halacha,Orach Chaim,Tefillah,Ten Minute Halacha,Title Ten Minute Halacha Greetings Before Dav...


In [ ]:
def __filter(merged, past: int = 7,filter: dict = None) -> pd.DataFrame:
        filtered_shiurim = __get_recent_interactions(merged, past)
        
        if filter is not None:
            for key, value in filter.items():
                filtered_shiurim = filtered_shiurim[filtered_shiurim[key] == value]
        
        return filtered_shiurim
    
def __get_recent_interactions(merged, past: int = 7) -> pd.DataFrame:
        interactions = merged
        today = pd.to_datetime(date.today())
        delta = today - timedelta(days=past)
        interactions = interactions[(interactions['date_played'] > delta) |
                                    (interactions['date_downloaded'] > delta) |
                                    (interactions['queue_date'] > delta)
                                    ]
        return interactions

In [ ]:
__filter(merged=merged, past=99999)

In [ ]:
from datetime import date, timedelta